In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder \
.master("local") \
.appName("groupby_sort_df") \
.getOrCreate()

In [3]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NV",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","DE",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","NV",80000,25,18000),
    ("Kumar","Marketing","NJ",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df1 = spark.createDataFrame(data=simpleData, schema = schema)

In [4]:
df1.groupBy("state").sum("salary").show()

+-----+-----------+
|state|sum(salary)|
+-----+-----------+
|   NJ|      91000|
|   NV|     166000|
|   CA|     171000|
|   DE|      99000|
|   NY|     252000|
+-----+-----------+



In [29]:
df1.groupBy("state") \
.agg(sum("salary").alias("sum_salary")).sort(desc("sum_salary")).where(col("sum_salary")>100000).show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NY|    252000|
|   CA|    171000|
|   NV|    166000|
+-----+----------+



In [30]:
df1.groupBy("state") \
.agg(sum("salary").alias("sum_salary")) \
.filter(col("sum_salary")>100000) \
.sort("sum_salary").show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NV|    166000|
|   CA|    171000|
|   NY|    252000|
+-----+----------+



In [31]:
df1.groupBy("state") \
.sum("salary") \
.withColumnRenamed("sum(salary)","sum_salary").show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NJ|     91000|
|   NV|    166000|
|   CA|    171000|
|   DE|     99000|
|   NY|    252000|
+-----+----------+



In [32]:
df1.createOrReplaceTempView("employee")
spark.sql("""select state, sum(salary) sum_salary from employee
group by state having sum_salary>100000 order by sum_salary desc""").show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NY|    252000|
|   CA|    171000|
|   NV|    166000|
+-----+----------+

